# Setup

In [1]:
from piper import piper
from piper.defaults import *
from piper.verbs import *
from piper.dates import to_date
from piper.factory import get_sample_data

piper version 0.0.7, last run: Friday, 26 February 2021 14:53:20


# Importing data into a dataframe

## read_csv()

Here's an example where we 'assign' the result to a variable (as in R)

In [2]:
%%piper
df <- pd.read_csv('inputs/2018 Bestsellers.csv') 
>> clean_columns()

## info()

In [3]:
%piper df >> info() >> pd.DataFrame.transpose()

Dataframe with (rows, cols) (1300, 9) consumes 0.41 Mb


,0,1,2,3,4,5,6,7,8
columns,title,week,list,rank,author,price,isbn,previous_rank,weeks_on_list
data_type,object,object,object,int64,object,float64,int64,float64,float64
unique,220,13,7,15,195,46,229,15,95
isna,0,0,0,0,0,0,0,514,390
isnull,0,0,0,0,0,0,0,514,390
total_count,1300,1300,1300,1300,1300,1300,1300,1300,1300


## select()

### ALL columns

select() allows you to select columns in various ways, default - ALL columns

In [4]:
# df = select(df, ['week', 'title'])
# df

In [5]:
%%piper
df >> select() >> head()

1300 rows, 9 columns


,title,week,list,rank,author,price,isbn,previous_rank,weeks_on_list
0,A Wrinkle in Time,2/14/2018,Early & Middle,1,Madeleine L'Engle,8.99,9781250153272,NaN,NaN
1,Wonder,2/14/2018,Early & Middle,2,R.J. Palacio,16.99,9781524720193,NaN,NaN
2,Auggie & Me,2/14/2018,Early & Middle,3,R.J. Palacio,16.99,9781101934852,NaN,NaN
3,The Girl Who Drank the Moon,2/14/2018,Early & Middle,4,Kelly Barnhill,16.95,9781616205676,NaN,NaN


### specific column(s)

Specify an column name, selects that column

In [6]:
%%piper
df >> select('title') >> head()

1300 rows, 1 columns


,title
0,A Wrinkle in Time
1,Wonder
2,Auggie & Me
3,The Girl Who Drank the Moon


passing a list retrieves the respective columns in one go.

In [7]:
%%piper
select(df, ['title', 'author', 'price']) >>
head()

1300 rows, 3 columns


,title,author,price
0,A Wrinkle in Time,Madeleine L'Engle,8.99
1,Wonder,R.J. Palacio,16.99
2,Auggie & Me,R.J. Palacio,16.99
3,The Girl Who Drank the Moon,Kelly Barnhill,16.95


### excluding column(s)

How about all fields EXCEPT certain columns?<br> No problem, specify a minus as a prefix to the column name.

In [8]:
%%piper
df
>> select(['-isbn', '-previous_rank'])
>> select('-weeks_on_list')
>> head()

1300 rows, 6 columns


,title,week,list,rank,author,price
0,A Wrinkle in Time,2/14/2018,Early & Middle,1,Madeleine L'Engle,8.99
1,Wonder,2/14/2018,Early & Middle,2,R.J. Palacio,16.99
2,Auggie & Me,2/14/2018,Early & Middle,3,R.J. Palacio,16.99
3,The Girl Who Drank the Moon,2/14/2018,Early & Middle,4,Kelly Barnhill,16.95


### selecting from one column to another column (column range/slice)

If you want to select a subset (range) of columns - pass a slice object with the from and to columns.

In [9]:
%piper df >> select(slice('title', 'isbn')) >> head()

1300 rows, 7 columns


,title,week,list,rank,author,price,isbn
0,A Wrinkle in Time,2/14/2018,Early & Middle,1,Madeleine L'Engle,8.99,9781250153272
1,Wonder,2/14/2018,Early & Middle,2,R.J. Palacio,16.99,9781524720193
2,Auggie & Me,2/14/2018,Early & Middle,3,R.J. Palacio,16.99,9781101934852
3,The Girl Who Drank the Moon,2/14/2018,Early & Middle,4,Kelly Barnhill,16.95,9781616205676


### combining rows containing header information

In [10]:
xl_file = Path('inputs/Data/sales_2017.xlsx')
temp_df = pd.read_excel(xl_file, header=None)
head(temp_df)

2155 rows, 12 columns


,0,1,2,3,4,5,6,7,8,9,10,11
0,Order,Customer,Sales,Order,Ship,Order,SKU,Order,Unit Sell,Discount,Shipping,Ship Mode
1,ID,ID,Person,Date,Date,Priority,Code,Quantity,Price,Percent,Amount,Container
2,13729,C508,Mr Robert Carlton,2017-01-01 00:00:00,2017-01-03 00:00:00,Not Specified,SKU947,9,95.99,0.08,35,Express Air - Large Box
3,28774,C372,Miss Roseanna Marr,2017-01-01 00:00:00,2017-01-02 00:00:00,High,SKU937,32,5.98,0.1,4.69,Regular Air - Small Box


In [11]:
%%piper
pd.read_excel(xl_file, header=None)
>> combine_header_rows()
>> head()

2153 rows, 12 columns


,Order ID,Customer ID,Sales Person,Order Date,Ship Date,Order Priority,SKU Code,Order Quantity,Unit Sell Price,Discount Percent,Shipping Amount,Ship Mode Container
2,13729,C508,Mr Robert Carlton,2017-01-01,2017-01-03,Not Specified,SKU947,9,95.99,0.08,35.00,Express Air - Large Box
3,28774,C372,Miss Roseanna Marr,2017-01-01,2017-01-02,High,SKU937,32,5.98,0.10,4.69,Regular Air - Small Box
4,9285,C212,Mr Robert Carlton,2017-01-02,2017-01-04,Critical,SKU363,3,40.98,0.06,2.99,Regular Air - Small Box
5,37537,C015,Mr Robert Carlton,2017-01-02,2017-01-02,Low,SKU052,4,291.73,0.00,48.80,Delivery Truck - Jumbo Drum


## count() / counts()

In [12]:
%piper df >> count() >> head()

9 rows, 2 columns


,index,n
0,title,1300
1,week,1300
2,list,1300
3,rank,1300


In [13]:
count(df.list)

,list,n
0,Hardcover Nonfiction,195
1,Hardcover Fiction,195
2,Early & Middle,195
3,Young Adult,195
4,Trade Paperback Nonfiction,195
5,Trade Paperback Fiction,195
6,Mass Market,130


In [14]:
%piper df >> count('list', add_total=True)

,index,n
0,Hardcover Nonfiction,195
1,Hardcover Fiction,195
2,Early & Middle,195
3,Young Adult,195
4,Trade Paperback Nonfiction,195
5,Trade Paperback Fiction,195
6,Mass Market,130
7,Total,1300


In [15]:
%%piper 
counts(df)

,index,n
0,title,1300
1,week,1300
2,list,1300
3,rank,1300
4,author,1300
5,price,1300
6,isbn,1300
7,previous_rank,786
8,weeks_on_list,910


In [16]:
%piper df >> counts(['list'], percent=True, cum_percent=True, reset_index=True)

,list,n,%,cum %
0,Early & Middle,195,15.0,15.0
1,Hardcover Fiction,195,15.0,30.0
2,Hardcover Nonfiction,195,15.0,45.0
3,Trade Paperback Fiction,195,15.0,60.0
4,Trade Paperback Nonfiction,195,15.0,75.0
5,Young Adult,195,15.0,90.0
6,Mass Market,130,10.0,100.0


In [17]:
%%piper 
count(df, 'weeks_on_list', bins=10)

,weeks_on_list,n
0,"(0.823, 18.6]",572
1,"(18.6, 36.2]",144
2,"(36.2, 53.8]",110
3,"(71.4, 89.0]",43
4,"(53.8, 71.4]",23
5,"(124.2, 141.8]",13
6,"(159.4, 177.0]",3
7,"(106.6, 124.2]",2
8,"(89.0, 106.6]",0
9,"(141.8, 159.4]",0


## assign()

In [18]:
%%piper
df 
>> select(slice('title', 'isbn'))
>> assign(week=lambda x: pd.to_datetime(x.week),
          price=lambda x: x.price.astype(str),
          concatenated_field = lambda x: x.title + ' ' + x.list, 
          price_premium = lambda x: (x.price.astype(float) * 4).round(2)) 
>> head(4)

1300 rows, 9 columns


,title,week,list,rank,author,price,isbn,concatenated_field,price_premium
0,A Wrinkle in Time,2018-02-14,Early & Middle,1,Madeleine L'Engle,8.99,9781250153272,A Wrinkle in Time Early & Middle,35.96
1,Wonder,2018-02-14,Early & Middle,2,R.J. Palacio,16.99,9781524720193,Wonder Early & Middle,67.96
2,Auggie & Me,2018-02-14,Early & Middle,3,R.J. Palacio,16.99,9781101934852,Auggie & Me Early & Middle,67.96
3,The Girl Who Drank the Moon,2018-02-14,Early & Middle,4,Kelly Barnhill,16.95,9781616205676,The Girl Who Drank the Moon Early & Middle,67.80


## to_date()

Alternative to using assign for date conversion 

In [19]:
%piper df >> to_date('week', format='%m/%d/%Y') >> head()

1300 rows, 9 columns


,title,week,list,rank,author,price,isbn,previous_rank,weeks_on_list
0,A Wrinkle in Time,2018-02-14,Early & Middle,1,Madeleine L'Engle,8.99,9781250153272,NaN,NaN
1,Wonder,2018-02-14,Early & Middle,2,R.J. Palacio,16.99,9781524720193,NaN,NaN
2,Auggie & Me,2018-02-14,Early & Middle,3,R.J. Palacio,16.99,9781101934852,NaN,NaN
3,The Girl Who Drank the Moon,2018-02-14,Early & Middle,4,Kelly Barnhill,16.95,9781616205676,NaN,NaN


## relocate()

In [20]:
%%piper
df >>
select(slice('title', 'isbn')) >>
assign(week=lambda x: pd.to_datetime(x.week),
       price=lambda x: x.price.astype(str),
       concatenated_field = lambda x: x.title + ' ' + x.list, 
       price_premium = lambda x: (x.price.astype(float) * 4).round(2))
>> relocate('author', loc='first')
>> relocate('isbn', loc='after', ref_column='week')
>> relocate(['concatenated_field', 'price_premium'], loc='before', ref_column='isbn')
>> head(4)

1300 rows, 9 columns


,author,title,week,concatenated_field,price_premium,isbn,list,rank,price
0,Madeleine L'Engle,A Wrinkle in Time,2018-02-14,A Wrinkle in Time Early & Middle,35.96,9781250153272,Early & Middle,1,8.99
1,R.J. Palacio,Wonder,2018-02-14,Wonder Early & Middle,67.96,9781524720193,Early & Middle,2,16.99
2,R.J. Palacio,Auggie & Me,2018-02-14,Auggie & Me Early & Middle,67.96,9781101934852,Early & Middle,3,16.99
3,Kelly Barnhill,The Girl Who Drank the Moon,2018-02-14,The Girl Who Drank the Moon Early & Middle,67.80,9781616205676,Early & Middle,4,16.95


## where()

### ==, between, isin, startswith, endswith, contains etc.

Note how you can mix and match where clauses with assignments and add additional where clauses when needed.

In [21]:
%%piper
df 
>> select(slice('title', 'isbn')) 
>> where(" author.str.strip() == 'R.J. Palacio' ")
>> assign(week=lambda x: pd.to_datetime(x.week),
          price=lambda x: x.price.astype(str))
>> where(" rank.between(2, 4) & price.astype('float').between(14, 17) ")
>> head(2)

20 rows, 7 columns


,title,week,list,rank,author,price,isbn
1,Wonder,2018-02-14,Early & Middle,2,R.J. Palacio,16.99,9781524720193
2,Auggie & Me,2018-02-14,Early & Middle,3,R.J. Palacio,16.99,9781101934852


In [22]:
%%piper
df >> select(slice('title', 'isbn'))
>> rename(columns={'list': 'list_'}) 
>> where(" list_.isin(['Early & Middle', 'Trade Paperback Nonfiction']) ") 
>> head(4)

390 rows, 7 columns


,title,week,list_,rank,author,price,isbn
0,A Wrinkle in Time,2018-02-14,Early & Middle,1,Madeleine L'Engle,8.99,9781250153272
1,Wonder,2018-02-14,Early & Middle,2,R.J. Palacio,16.99,9781524720193
2,Auggie & Me,2018-02-14,Early & Middle,3,R.J. Palacio,16.99,9781101934852
3,The Girl Who Drank the Moon,2018-02-14,Early & Middle,4,Kelly Barnhill,16.95,9781616205676


In [23]:
%%piper
df >> select(slice('title', 'isbn'))
>> rename(columns={'list': 'list_'}) 
>> where(" list_.str.startswith('Early') | list_.str.endswith('ion') ") 
>> head(2)

975 rows, 7 columns


,title,week,list_,rank,author,price,isbn
0,A Wrinkle in Time,2018-02-14,Early & Middle,1,Madeleine L'Engle,8.99,9781250153272
1,Wonder,2018-02-14,Early & Middle,2,R.J. Palacio,16.99,9781524720193


In [24]:
%%piper
df >> select(slice('title', 'isbn'))
>> rename(columns={'list': 'list_'}) 
>> where(" list_.str.lower().str.contains('young') ")
>> head(4)

195 rows, 7 columns


,title,week,list_,rank,author,price,isbn
85,The Hate U Give,2018-02-14,Young Adult,1,Angie Thomas,17.99,9780062498533
86,Turtles All the Way Down,2018-02-14,Young Adult,2,John Green,19.99,9780525555360
87,Tempests and Slaughter,2018-02-14,Young Adult,3,Tamora Pierce,18.99,9780375847110
88,Every Day,2018-02-14,Young Adult,4,David Levithan,10.99,9780525581611


## distinct()

In [25]:
%piper df >> distinct('isbn') >> head()

229 rows, 9 columns


,title,week,list,rank,author,price,isbn,previous_rank,weeks_on_list
0,A Wrinkle in Time,2018-02-14,Early & Middle,1,Madeleine L'Engle,8.99,9781250153272,NaN,NaN
1,Wonder,2018-02-14,Early & Middle,2,R.J. Palacio,16.99,9781524720193,NaN,NaN
2,Auggie & Me,2018-02-14,Early & Middle,3,R.J. Palacio,16.99,9781101934852,NaN,NaN
3,The Girl Who Drank the Moon,2018-02-14,Early & Middle,4,Kelly Barnhill,16.95,9781616205676,NaN,NaN


## group_by()

In [26]:
%%piper
df 
>> group_by(['title', 'author'])
>> summarise(mean_price=('price', 'mean')) 
>> assign(mean_price_times_something=lambda x: x.mean_price * 3) 
>> order_by(['author', 'title'], ascending=[False, False])
>> head(5).reset_index()

225 rows, 2 columns


,title,author,mean_price,mean_price_times_something
0,Feel Free,Zadie Smith,28.00,84.00
1,Leonardo da Vinci,Walter Isaacson,35.00,105.00
2,The Sympathizer,Viet Thanh Nguyen,16.00,48.00
3,Roller Girl,Victoria Jamieson,12.99,38.97
4,All's Faire in Middle School,Victoria Jamieson,12.99,38.97


### compress rows to a list / explode list

In [27]:
%%piper
df >> group_by('list')
>> summarise({'author': lambda x: x.tolist()})
# >> explode('author')  
>> head(6)

7 rows, 1 columns


,author
list,
Early & Middle,"[Madeleine L'Engle, R.J. Palacio, R.J. Palacio, Kelly Barnhill, Raina Telgemeier, Kimberly Brubaker Bradley, Raina T..."
Hardcover Fiction,"[Kristin Hannah, Tayari Jones, A.J. Finn, Amor Towles, Chloe Benjamin, Celeste Ng, Jesmyn Ward, Jojo Moyes, Dan Brow..."
Hardcover Nonfiction,"[Michael Wolff, Neil deGrasse Tyson, Mark Manson, Walter Isaacson, Tiffany Haddish, Dave Eggers, David Grann, Pete S..."
Mass Market,"[Lee Child, Ernest Cline, John Grisham, James Patterson, Michael Crichton, Liane Moriarty, Danielle Steel, Brian Kil..."
Trade Paperback Fiction,"[Colson Whitehead, George Saunders, Min Jin Lee, Jessica Shattuck, Ruth Ware, Georgia Hunter, Fredrik Backman, Antho..."
Trade Paperback Nonfiction,"[Jen Sincero, Peter Frankopan, Doug Stanton, Jonah Berger, Atul Gawande, Douglas J. Preston, Harvard Business School..."


## summarise()

In [28]:
%%piper 
get_sample_data() 
>> select('-dates')
>> summarise({'values_1': 'sum', 'values_2': 'sum'})

values_1    73604
values_2    75163
dtype: int64

In [29]:
%%piper 
get_sample_data() 
>> select('-dates')
>> where("regions.isin(['North', 'South'])")
>> group_by(['regions', 'countries'])
>> summarise({'values_1': 'sum', 'values_2': 'sum'})
>> rename(columns={'values_1': 'totval1', 'values_2': 'totval2'})
>> assign(pcent_val1 = lambda x: (x.totval1 * 100/x.totval1.sum()).round(2),
           grp_pcent = lambda x: x.groupby(['regions'])['totval2'].transform(lambda x: (x*100/x.sum()).round(2))
         )
>> head(10)

16 rows, 4 columns


totval1  totval2  pcent_val1  grp_pcent
regions countries                                           
North   France          2275     2926        6.43      16.29
        Germany         2239     1736        6.33       9.67
        Italy           1868     1926        5.28      10.72
        Norway          2633     3282        7.44      18.27
        Portugal        1857     1861        5.25      10.36
        Spain           2447     2457        6.92      13.68
        Sweden          1535     1417        4.34       7.89
        Switzerland     2723     2356        7.70      13.12
South   France          2118     1726        5.99       9.66
        Germany         1753     2028        4.96      11.35

## pivot_table()

In [30]:
head(df)

1300 rows, 9 columns


,title,week,list,rank,author,price,isbn,previous_rank,weeks_on_list
0,A Wrinkle in Time,2018-02-14,Early & Middle,1,Madeleine L'Engle,8.99,9781250153272,NaN,NaN
1,Wonder,2018-02-14,Early & Middle,2,R.J. Palacio,16.99,9781524720193,NaN,NaN
2,Auggie & Me,2018-02-14,Early & Middle,3,R.J. Palacio,16.99,9781101934852,NaN,NaN
3,The Girl Who Drank the Moon,2018-02-14,Early & Middle,4,Kelly Barnhill,16.95,9781616205676,NaN,NaN


In [31]:
index = ['list']
values=['rank', 'price']

In [32]:
%%piper

df
>> pivot_table(index=index, values=values, margins=True, fill_value=0,
               aggfunc={'rank': 'count', 'price': np.mean})
>> rename(columns={'price': 'mean_price', 'rank': 'total_rows'})
>> assign(mean_price=lambda x: x.mean_price.round(2)).reset_index()
>> head(4)

8 rows, 3 columns


,list,mean_price,total_rows
0,Early & Middle,13.53,195
1,Hardcover Fiction,27.25,195
2,Hardcover Nonfiction,28.20,195
3,Mass Market,9.91,130
